#  LLMRouterChain

### Let's say you are a Computer Scientist and has advanced level of knowledge of Computer Science, but you are studying basic physics to teach to your kids, so you want a LLM who answers the questions accordingly. If you ask it a computer science question it should answer like a professor including technical and complex details and When you ask it a physics question it answers in a simple understandable language.

### Route Templates

In [1]:
beginner_template = '''You are a physics teacher who is really
focused on beginners and explaining complex topics in simple to understand terms. 
You assume no prior knowledge. Here is the question\n{input}'''

In [2]:
expert_template = '''You are a world expert Computer Science professor who explains Computer science topics
to advanced audience members. You can assume anyone you answer has a 
PhD level understanding of Computer Science. Here is the question\n{input}'''

In [3]:
# ADD YOUR OWN TEMPLATES !
empty_template = 'empty'

### Route Prompts

In [4]:
prompt_infos = [
    {'name':'empty','description':'Replies to empty questions','prompt_template':empty_template},
    {'name':'advanced computer science','description': 'Answers advanced computer science questions',
     'prompt_template':expert_template},
    {'name':'beginner physics','description': 'Answers basic beginner physics questions',
     'prompt_template':beginner_template},
    
]

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from dotenv import load_dotenv, find_dotenv
import os

load_dotenv(find_dotenv(), override=True)
#In the .env file, save OPENAI_API_KEY = {} of the openAI API account
api_key = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(model_name="gpt-3.5-turbo",openai_api_key=api_key)

In [6]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

In [7]:
# destination_chains

In [8]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm,prompt=default_prompt)

### Routing Destinations


In [9]:
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [10]:
print(destinations_str)

empty: Replies to empty questions
advanced computer science: Answers advanced computer science questions
beginner physics: Answers basic beginner physics questions


### Router Prompt

In [13]:
from langchain.prompts import PromptTemplate
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [14]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

In [15]:
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
empty: Replies to empty questions
advanced computer science: Answers advanced computer science questi

### Routing Chain Call

In [16]:
from langchain.chains.router import MultiPromptChain

In [17]:
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [18]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [19]:
chain.run("How do magnets work?")



> Entering new MultiPromptChain chain...


C:\Users\yashnarendra\AppData\Local\anaconda3\Lib\site-packages\langchain\chains\llm.py:321: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


beginner physics: {'input': 'How do magnets work?'}
> Finished chain.


'Magnets are fascinating objects that have the ability to attract or repel certain materials. They work because of a property called magnetism. \n\nTo understand how magnets work, let\'s start by understanding what atoms are. Everything around us, including magnets, is made up of tiny particles called atoms. Atoms have even smaller particles inside them, such as protons and electrons.\n\nNow, imagine that these electrons are like tiny spinning tops. When electrons spin, they create a magnetic field around them. Think of this field as an invisible force field that surrounds the electron.\n\nIn most materials, the electrons spin in random directions, canceling out their magnetic fields. But in certain materials, like iron or nickel, the atoms arrange themselves in a way that causes the electrons to align their spins in the same direction. This alignment creates a stronger magnetic field.\n\nWhen you bring two magnets close to each other, their magnetic fields interact. If the magnets are

In [20]:
chain.run("What is race condition")



> Entering new MultiPromptChain chain...


C:\Users\yashnarendra\AppData\Local\anaconda3\Lib\site-packages\langchain\chains\llm.py:321: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


advanced computer science: {'input': 'What is a race condition?'}
> Finished chain.


'A race condition is a phenomenon that occurs in concurrent computing, where the behavior or outcome of a system is dependent on the relative timing or sequencing of events. It arises when multiple threads or processes access shared resources or execute certain operations concurrently, leading to unexpected and non-deterministic outcomes.\n\nIn computer science, concurrent execution refers to situations where multiple threads of execution or processes are executing in an overlapping manner, attempting to access and modify shared data simultaneously. Race conditions occur when the correctness of the program\'s logic or the integrity of shared resources become compromised due to the unpredictable interleaving of operations.\n\nThe term "race condition" is derived from the idea that multiple threads or processes are racing to access or modify a shared resource, and the final result of the program or system depends on which thread or process "wins" the race.\n\nRace conditions can manifest

In [21]:
chain.run("What is recipe of white souce pasta condition")



> Entering new MultiPromptChain chain...


C:\Users\yashnarendra\AppData\Local\anaconda3\Lib\site-packages\langchain\chains\llm.py:321: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


None: {'input': 'What is recipe of white souce pasta condition'}
> Finished chain.


'Here is a recipe for white sauce pasta:\n\nIngredients:\n- 8 ounces of pasta (penne, fettuccine, or any other shape you prefer)\n- 2 tablespoons of butter\n- 2 tablespoons of all-purpose flour\n- 2 cups of milk\n- 1/2 cup of grated Parmesan cheese\n- Salt and pepper to taste\n- Optional: garlic powder, dried herbs (such as oregano or basil), and red pepper flakes for additional flavor\n\nInstructions:\n1. Cook the pasta according to the package instructions until al dente. Drain and set aside.\n\n2. In a separate saucepan, melt the butter over medium heat.\n\n3. Once the butter has melted, whisk in the flour until smooth, creating a roux. Cook the roux for about 1-2 minutes, stirring constantly.\n\n4. Gradually pour in the milk, whisking continuously to avoid any lumps. Continue cooking and stirring until the sauce thickens, usually around 5-7 minutes.\n\n5. Add the grated Parmesan cheese to the sauce, stirring until it melts completely. If desired, add garlic powder, dried herbs, and